In [1]:
import os

import pandas as pd
from data_loader import label_str2int_dict

rationale_names = ['td','itc','img','cs']
print(os.getcwd())
os.chdir('/home/lyq/DataSet/FakeNews/weibo_dataset')
print(os.getcwd())


/home/lyq/PycharmProjects/QwenVLRationaleGenerate/data/weibo
/home/lyq/DataSet/FakeNews/weibo_dataset


In [2]:
import ast

weibo_df = pd.read_csv('weibo.csv')
weibo_df = weibo_df.rename(columns={
    'id':'source_id',
    'text':'content',
})

def get_image_id(image_file_name_list):
    return ast.literal_eval(image_file_name_list)[0].split('/')[-1].split('.')[0]


weibo_df['content'] = weibo_df.apply(lambda x : f'{x["content"]} --发布来源：{x["release_source"]}', axis=1)
weibo_df['label'] = weibo_df['label'].apply(lambda x : label_str2int_dict[x])
weibo_df['image_id'] = weibo_df['available_image_paths'].apply(lambda x : get_image_id(x))

weibo_df = weibo_df.drop(columns=['available_image_paths','release_source'])

weibo_df

,source_id,content,split,label,image_id
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,train,1,a71ac854gw1dytin2zmk9j
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,train,1,3ba161e7jw1e4i6j4ep61j211n0r4juz
2,3899073935617462,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",train,1,006ajvCngw1ex4jjl2pwlj30c8096wf7
3,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",train,1,6112c76ajw1e591p39mpej20hw094mxh
4,3553661986467439,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,train,1,a5404163jw1e2iljwbrztj
...,...,...,...,...,...
7956,3896157270699471,【空城记：刘小东与鄂尔多斯】#正午故事#鄂尔多斯是个比较敏感的城市。其实政府也比想象的聪明，...,test,0,005EHRy1jw1ewv1azxwh1j30rs0lljzm
7957,3895644894378737,【日本人是怎么拍黑帮电影的？】Tim Gallo出生于俄国，17岁后迁往日本生活。通过用镜头...,test,0,005EHRy1jw1ewtlhknf30j30jg0czq4y
7958,3893257346088858,#跟着面叔看世界#第一站，日本东京筑地市场。你现在到哪儿了？不会还堵在路上吧？一起来晒晒旅途...,test,0,005EHRy1jw1ewlzj6oenaj30qo0zkdm5
7959,3892828281041275,【现场图·广西柳城县连续发生多起爆炸】据@南宁电视台 ：网友称，现场听到几声巨响，车站，市场...,test,0,61e04755jw1ewkm8o7fyyj20c80gb0u2


In [3]:
data_set_len = weibo_df.shape[0]

def findNoRationaleNews(df,r_df):
    return set(df['source_id'].tolist()) - set(r_df['source_id'].tolist())

for rationale_name in rationale_names:
    r_df = pd.read_csv(f'{rationale_name}.csv')
    r_df = r_df.dropna(subset=['authenticity','reason'])
    r_df['authenticity'] = r_df['authenticity'].apply(lambda x : label_str2int_dict[x])
    if r_df.shape[0] != data_set_len:
        raise Exception(f'{rationale_name} len : {r_df.shape[0]} != {data_set_len} , No Rationale id : {findNoRationaleNews(weibo_df,r_df)}')
    weibo_df = weibo_df.merge(r_df,how='left',on='source_id')
    weibo_df = weibo_df.rename(columns={
        'authenticity':f'{rationale_name}_pred',
        'reason': f'{rationale_name}_rationale'
    })
    weibo_df[f'{rationale_name}_acc'] = weibo_df.apply(lambda x : int(x[f'{rationale_name}_pred'] == x['label']), axis=1)


weibo_df

Exception: img len : 4537 != 7961 , No Rationale id : {3531190344220672, 3923377129160704, 3865184932462592, 3760221413302275, 3875380035805193, 3854476437692426, 3861440991608843, 3852517349818383, 3844488151498769, 3833353688195090, 3501836730245140, 3536190931763221, 3849967897042967, 3918914491113496, 3544557058465817, 3885460361150490, 3918133821210651, 3723163332231193, 3526368157155355, 3924335947014175, 3863392026247201, 3929099741036579, 3720713632849956, 3927124064763950, 3919210776313903, 3915739050238000, 3927697111203897, 3927944944672827, 3927857287471164, 3467177711706173, 3653666680823872, 3766334578180165, 3896368076316741, 3925380077166670, 3914601555320911, 3908802090524752, 3919913397690450, 3911393458241618, 3801635740639316, 3922975482962005, 3913187738591318, 3834078312980569, 3748170577952858, 3464577842643036, 3917459989635166, 3906136757051488, 3491018185580641, 3689147518296166, 3508359401898087, 3902551906746472, 3757538845474921, 3840705401675884, 391989971

In [4]:

from Util import cal_rationale_metrics

for rationale_name in rationale_names:
    print(f'{rationale_name} metrics: \n '
          f'{cal_rationale_metrics(weibo_df[f"{rationale_name}_pred"].to_numpy(),weibo_df["label"].to_numpy())}')


td metrics: 
 {'acc': 0.8233890214797136, 'recall': 0.8141402388002694, 'recall_real': 0.7053816584294343, 'recall_fake': 0.9228988191711044, 'precision': 0.8424585086361885, 'precision_real': 0.8929440389294404, 'precision_fake': 0.7919729783429367, 'f1_macro': 0.8202978370390899, 'f1_real': 0.7881576928976837, 'f1_fake': 0.8524379811804962}
itc metrics: 
 {'acc': 0.8015324707951262, 'recall': 0.7873686299086613, 'recall_real': 0.6208127402526085, 'recall_fake': 0.953924519564714, 'precision': 0.8428454966033521, 'precision_real': 0.930835734870317, 'precision_fake': 0.754855258336387, 'f1_macro': 0.7938234455065132, 'f1_real': 0.7448525778290233, 'f1_fake': 0.8427943131840032}
img metrics: 
 {'acc': 0.6153749528953649, 'recall': 0.6356637891980558, 'recall_real': 0.8742449203734212, 'recall_fake': 0.39708265802269044, 'precision': 0.6731564132117969, 'precision_real': 0.5530658328990794, 'precision_fake': 0.7932469935245143, 'f1_macro': 0.6033790999541508, 'f1_real': 0.67751888498776

In [5]:

from sklearn.model_selection import train_test_split


def split_data(df, train_ratio=0.8, valid_ratio=0.1, random_state=None, stratify_col=None):
    """
    将 pandas DataFrame 划分为训练集、验证集和测试集。

    参数:
        df (pd.DataFrame): 包含所有数据的 pandas DataFrame。
        train_ratio (float): 训练集所占比例，默认为 0.8。
        valid_ratio (float): 验证集所占比例，默认为 0.1。
        random_state (int, RandomState instance or None): 控制随机抽样的种子，默认为 None。
        stratify_col (str or None): 如果不为 None，则数据将按照指定列进行分层抽样，默认为 None。

    返回:
        tuple: 包含训练集、验证集和测试集的元组。
               每个集合都是原始 DataFrame 的一部分，包含所有原始列。
    """
    # 计算测试集的比例
    test_ratio = 1 - train_ratio - valid_ratio
    if test_ratio <= 0:
        raise ValueError("The sum of train_ratio and valid_ratio must be less than 1.")

    # 分层抽样的依据列（如果指定了）
    stratify = df[stratify_col] if stratify_col else None

    # 第一步：先从全部数据中划分出测试集
    remaining_df, test_df = train_test_split(
        df,
        test_size=test_ratio ,
        random_state=random_state,
        shuffle=True,
        stratify=stratify
    )

    # 更新分层抽样的依据列（如果指定了）
    stratify_remaining = remaining_df[stratify_col] if stratify_col else None

    # 第二步：从剩余的数据中划分出训练集和验证集
    train_df, valid_df = train_test_split(
        remaining_df,
        test_size=valid_ratio / (train_ratio + valid_ratio),
        random_state=random_state,
        shuffle=True,
        stratify=stratify_remaining
    )

    return train_df, valid_df, test_df

def count_dataset(df):
    return {
        'sum': df.shape[0],
        'real':(df['label']==label_str2int_dict['real']).sum(),
        'fake':(df['label']==label_str2int_dict['fake']).sum(),
    }

train_df,val_df,test_df = split_data(weibo_df,train_ratio=0.6,valid_ratio=0.2,random_state=666)
train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'
print(f'train : {count_dataset(train_df)}')
print(f'val : {count_dataset(val_df)}')
print(f'test : {count_dataset(test_df)}')


train : {'sum': 4776, 'real': 2161, 'fake': 2615}
val : {'sum': 1592, 'real': 741, 'fake': 851}
test : {'sum': 1593, 'real': 740, 'fake': 853}


In [6]:
weibo_df.to_csv('weibo_llm_rationale.csv',index=False)
train_df.to_csv('train.csv',index=False)
val_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)